In [ ]:
##########==========##########==========##########==========##########==========

In [ ]:
print('TODO: PPTX13 code is SLOPPY quick code; will need to be rewritten ASAP')

## HEAD

#### HEAD01 - load libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import matplotlib

def define_style():
    plt.rcParams['font.family'] = 'sans-serif'
    plt.rcParams['font.sans-serif'] = ['Quicksand']
    plt.rcParams['font.weight'] = 'medium'
    plt.rcParams['font.size'] = 12
    plt.rcParams['ytick.labelsize'] = 12
    plt.rcParams['xtick.labelsize'] = 12
    plt.rcParams['axes.labelsize'] = 12
    
define_style()

#### HEAD02 - load data files

In [ ]:
def load_pickle(addr):
    conn = open(addr, 'rb')
    the_object = pickle.load(conn)
    conn.close()
    return the_object

handle_data = pd.read_excel('A_Input/twitter_handles.xlsx')
user_data = pd.read_csv('B_Process/user_data.csv')
tweet_data = pd.read_csv('B_Process/tweet_data.csv')
best_model_stats = load_pickle('B_Process/best_model_stats.pickle')


## load hparam files
logistic_hparams = load_pickle('B_Process/models/logistic_hparams.pickle')
bayes_hparams    = load_pickle('B_Process/models/bayes_hparams.pickle')
forest_hparams   = load_pickle('B_Process/models/forest_hparams.pickle')
adaboost_hparams = load_pickle('B_Process/models/adaboost_hparams.pickle')

## PPTX

#### PPTX07 - Data Sources and Collection (Slide 07) - Calculate data collection statistics

In [ ]:
## calculate data collection statistics
def calculate_collection_stats(
    hd = handle_data, ud = user_data, td = tweet_data):
    
    ## define useful indices
    congress_idx = ['USA House', 'USA Senate']
    
    ## calculate handle_data statistics
    hd_stats = {
        'Total': hd.shape[0],
        'Congress': hd.group.value_counts().loc[congress_idx].sum(),
        'Republican': hd.party.value_counts().loc['Republican']
    }
    
    ## calculate user statistics
    include_idx = ud.ml_set != 'exclude'
    ud_stats = {
        'Total': ud.loc[include_idx].shape[0],
        'Congress': ud.group.loc[include_idx].value_counts().loc[
            congress_idx].sum(),
        'Republican': ud.party.loc[include_idx].value_counts().loc['Republican']
    }
    
    ##calcualte tweet statistics
    td_stats = {
        'Total': td.shape[0],
        'Congress': td.merge(ud, how = 'left', right_on = 'handle',
            left_on = 'screen_name').group.value_counts().loc[congress_idx].sum(),
        'Republican': td.merge(ud, how = 'left', right_on = 'handle',
            left_on = 'screen_name').party.value_counts().loc['Republican'].sum()
    }
    
    ## compile statistics
    col_stats = {'Handles': hd_stats, 'Users': ud_stats, 'Tweets': td_stats}
    
    return pd.DataFrame(col_stats).T


calculate_collection_stats()

#### PPTX12 - Model Performance (Slide 12)

In [ ]:
cp = {
    'AzureDark': '#021F33',
    'BrownDark': '#804419',
    'AzureFaded': '#466F91',
    'BrownFaded': '#916846',
    'AzureBg': '#CAD9E6',
    'BrownBg': '#E6D6CA',
    'AzureGrid': '#B4C1CC',
    'Trans': '#11223300'
    }
cp['AzureBgTrans'] = cp['AzureBg'] + '40'

## prepare data
best_model_stats[('time', 'run_time')] = (best_model_stats[('time', 'run_time')
        ]).fillna(min(best_model_stats[('time', 'run_time')].values))
best_model_stats = best_model_stats.reset_index()

## drop staged random guess model
i = best_model_stats[('model','variant')] != 'stage'
best_model_stats = best_model_stats.loc[i]

## configure plotting parameters to represent different model
best_model_stats[('label', 'model')] = best_model_stats[(
    'model', 'algorithm')].replace({'bayes':'NB', 'forest':'RF',
    'logistic':'LR', 'adaboost':'AB', 'random':'RG'})

bm_col = {
    'feature_unitary': cp['BrownDark'],
    'feature_stage': cp['AzureDark'],
    'pca_unitary': cp['BrownFaded'],
    'pca_stage': cp['AzureFaded'],
    'unitary': cp['BrownDark'],
    'stage': cp['AzureDark'],
}

best_model_bbox = {
    'feature_unitary': dict(boxstyle = 'round', linestyle = '-',
            facecolor = cp['AzureBg'],
            edgecolor = bm_col['feature_unitary'] ),
    'feature_stage': dict(boxstyle = 'round', linestyle = '-',
            facecolor = cp['AzureBg'],
            edgecolor = bm_col['feature_stage']),
    'pca_unitary': dict(boxstyle = 'round', linestyle = ':',
            facecolor = cp['Trans'],
            edgecolor = bm_col['pca_unitary']),
    'pca_stage': dict(boxstyle = 'round', linestyle = ':',
            facecolor = cp['Trans'],
            edgecolor = bm_col['pca_stage']),
    'unitary': dict(boxstyle = 'round', linestyle = '-',
            facecolor = cp['Trans'],
            edgecolor = bm_col['unitary']),
    'stage': dict(boxstyle = 'round', linestyle = '-',
            facecolor = cp['Trans'],
                  edgecolor = bm_col['stage']),
    }

In [ ]:
## generate f1 x negative runtime performance plot
plt.scatter(
    x = best_model_stats[('time', 'run_time')],
    y = best_model_stats[('perform', 'f1')],
    c = '#FFFFFF00'
)

best_model = ['feature_unitary', 'unitary']

## render non-best model variants
for i in best_model_stats.index:
    if best_model_stats.loc[i, ('model', 'variant')] not in best_model:
        plt.text(
            x = best_model_stats.loc[i, ('time', 'run_time')],
            y = best_model_stats.loc[i, ('perform', 'f1')],
            s = best_model_stats.loc[i, ('label', 'model')],
            bbox = best_model_bbox[best_model_stats.loc[i, ('model', 'variant')]],
            c = bm_col[best_model_stats.loc[i, ('model', 'variant')]]
    )

## render best model variant
for i in best_model_stats.index:
    if best_model_stats.loc[i, ('model', 'variant')] in best_model:
        plt.text(
            x = best_model_stats.loc[i, ('time', 'run_time')],
            y = best_model_stats.loc[i, ('perform', 'f1')],
            s = best_model_stats.loc[i, ('label', 'model')],
            bbox = best_model_bbox[best_model_stats.loc[i, ('model', 'variant')]],
            c = bm_col[best_model_stats.loc[i, ('model', 'variant')]]
    )

import matplotlib.patches as mpatches
        
the_handles = [
    mpatches.Patch(edgecolor =  bm_col['feature_unitary'],
        label='One-Stage, Features', facecolor = cp['AzureBgTrans'], 
                   linestyle = '-'),
    mpatches.Patch(edgecolor = bm_col['pca_unitary'], label='One-Stage, PCA',
        facecolor = cp['AzureBgTrans'], linestyle = ':'),
    mpatches.Patch(edgecolor = bm_col['feature_stage'],
                   label='Two-Stage, Features',
        facecolor = cp['AzureBgTrans'], linestyle = '-'),
    mpatches.Patch(edgecolor=bm_col['pca_stage'], label='Two-Stage, PCA',
        facecolor = cp['AzureBgTrans'], linestyle = ':')
]

plt.legend(handles = the_handles,
           facecolor = cp['AzureBg'], edgecolor = '#00000000')


def set_plot_parameters():
    matplotlib.rcParams['figure.edgecolor'] = cp['AzureDark']
    matplotlib.rcParams['axes.edgecolor'] = cp['AzureDark']
    matplotlib.rcParams['axes.labelcolor'] = cp['AzureDark']
    plt.gcf().set_size_inches(4.5, 4.5)
    plt.gcf().set_facecolor(cp['AzureBg'])
    plt.gca().set_facecolor(cp['AzureBg'])
    plt.gca().tick_params(axis='x', colors = cp['AzureDark'])
    plt.gca().tick_params(axis='y', colors = cp['AzureDark'])

## plot parameters


set_plot_parameters()
plt.gca().set_xlabel('Seconds of runtime (logged) • Model is fast ---->')
plt.gca().set_ylabel('F1 score on test dataset • Model is accurate ---->')
plt.gca().set_xlim((0.90, 2**11))
plt.gca().set_ylim(0.3, 1.005)
plt.gca().set_xscale('log', base = 2)
plt.gca().invert_xaxis()
plt.grid(color = cp['AzureGrid'])

plt.gcf().savefig('C_Output/model_performance.pdf')

#### PPTX11 - Hyperparameter Testing (Slide 11)

In [ ]:
## extract scoring information from hyperparameter search objects
def extract_test_scores(hparams, pca = False):
    
    ## extract scores
    result = dict()
    for i in hparams.keys():
        if pca:
            if i[0:3] == "pca":
                result[i] = hparams[i].cv_results_["mean_test_score"]
        else:
            if i[0:3] != "pca":
                result[i] = hparams[i].cv_results_["mean_test_score"]
            
    ## package and return
    result = pd.DataFrame(result)
    return result.values.reshape(-1, 1)

def compile_test_scores(hparam_dict = {
        'LR': logistic_hparams,
        'NB': bayes_hparams,
        'RF': forest_hparams,
        'AB': adaboost_hparams,
        }, get_pca = False):
    result = dict()
    max_length = 0
    for i in hparam_dict.keys():
        result[i] = pd.DataFrame(
            extract_test_scores(hparam_dict[i], pca = get_pca))
        result[i] = result[i].squeeze().sample(n = int(1e5), replace = True,
                                     random_state = 4561)
        result[i] = result[i].reset_index(drop = True)
        
    result = pd.DataFrame(result)

    return result

## execute code
hparams = compile_test_scores(get_pca = False)
hparams_pca = compile_test_scores(get_pca = True)
hparams = pd.concat([hparams, hparams_pca], axis = 1)

In [ ]:
## generate boxplots to represent spread of accuracy scores given parameters

set_plot_parameters()
the_box = plt.boxplot(hparams, labels = hparams.columns, 
            patch_artist=True,
            boxprops = dict(
                edgecolor = cp["AzureDark"], facecolor = cp['AzureBg']),
            whiskerprops = dict(color = cp["AzureDark"]),
            capprops = dict(color = cp["AzureDark"]),
            flierprops = dict(color = cp["AzureDark"]),
            medianprops = dict(color = cp["AzureDark"]),
            showfliers = False,
            whis = (10, 90)
            
            )
plt.grid(color = cp['AzureGrid'])
plt.gca().set_ylim(0.3, 1)
plt.gca().set_ylabel('F1 score on test dataset • Model is accurate ---->')
plt.gca().set_xlabel(loc = 'left',
    xlabel = ' ' * 11 +'Features' + ' ' * 13 + 'Principal Components')
plt.gcf().savefig('C_Output/hyperparameter_performance.pdf')

## FOOT - display objects as needed